In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

In [ ]:
links = fetch_website_links("https://ewelina-beben.profesjonalnyprogramista.pl/")
links

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://ewelina-beben.profesjonalnyprogramista.pl/"))

In [ ]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [ ]:
select_relevant_links("https://ewelina-beben.profesjonalnyprogramista.pl/")

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://ewelina-beben.profesjonalnyprogramista.pl/")

In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://ewelina-beben.profesjonalnyprogramista.pl/"))

In [ ]:
coverletter_system_prompt = """
You are an assistant that writes a professional and convincing cover letter (Motivationsschreiben)
in German for a job application.

The letter should be written from the perspective of a candidate applying for a Junior Developer
(or Junior Software Developer) position.

Guidelines:
- Write in correct, natural German (formal tone, business-appropriate).
- Emphasize motivation to learn, basic programming skills, interest in software development,
  and willingness to grow professionally.
- Mention enthusiasm for technology, teamwork, and problem-solving.
- Keep the letter concise (approx. 3–4 paragraphs).
- Do not invent very advanced experience; focus on junior-level potential.
- Do not use bullet points or headings — write a proper letter text.

Return only the cover letter text, without explanations or code blocks.
"""


In [ ]:
def get_coverletter_user_prompt(company_name, url, personal_website):
    user_prompt = f"""
You are preparing a job application for a company called: {company_name}.

Below you will find the contents of the company's website and other relevant pages.
Use this information to write a personalized cover letter (Motivationsschreiben)
in German for a Junior Developer position.

Candidate information:
- Personal website / portfolio: {personal_website}

Instructions:
- Address the company directly.
- Naturally include a reference to the candidate’s personal website.
- Refer to the company’s values, products, or technologies if available.
- Write in a professional, formal German tone.
- Output only the cover letter text.

Company information:
\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [ ]:
get_coverletter_user_prompt("Goldbeck", "https://goldbeck.de", "https://ewelina-beben.profesjonalnyprogramista.pl/")

In [ ]:
def create_coverletter(company_name, url, personal_website):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": coverletter_system_prompt},
            {"role": "user", "content": get_coverletter_user_prompt(company_name, url, personal_website)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_coverletter("Goldbeck", "https://goldbeck.de", "https://ewelina-beben.profesjonalnyprogramista.pl/")

In [ ]:
def stream_coverletter(company_name, url, personal_website):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": coverletter_system_prompt},
            {"role": "user", "content": get_coverletter_user_prompt(company_name, url, personal_website)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_coverletter("Goldbeck", "https://goldbeck.de", "https://ewelina-beben.profesjonalnyprogramista.pl/")